In [1]:
#Farklı bir classificcation problem ile karşılaştığında yapılacak düzeltmeler
#1. .npz nin ismini değiştir.
#2. classes_num ı değiştir.

import numpy as np
# Batching yapacak bir class yapıyoruz

class Audiobooks_Data_Reader():
    #Bu class iterator olacak. 
    
    #init .npz dosyasındaki datayı yüklüyor.
    def __init__(self,dataset,batch_size=None):
        #Batch size optional. girilmezse none girilirse o olur. Validation ve test setlerinde batch e gerek olmadığından onlarda none olabilir.
        #x('train',5) dediğimizde Audiobooks_data_train.npz yi batch size 5 olacak şekilde yükler.
        
        npz=np.load("Audiobooks_data_{0}.npz".format(dataset))
        self.inputs, self.targets= npz["inputs"].astype(np.float), npz["targets"].astype(np.int)
        
        #batch size verilmemişse tek batch yap dedik.
        if batch_size is None:
            self.batch_size=self.inputs.shape[0]
        else:
            self.batch_size=batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
    
    #next methodu npz deki bir sonraki batchi yükler.
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
        
        #datayı batchlere böldük ve next bize bunları sırayla yüklemeyi sağlar.
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)#hangi parçaların alınacağı. (0*batch_size dan 1*batcsize a kadar böl diyor.)
        inputs_batch = self.inputs[batch_slice]
        targets_batch  = self.targets[batch_slice]
        self.curr_batch +=1

        #one-hot encode the targets. zaten 0-1 lerden oluşuyor ama sonraki kullanımlar için böyle yaptık.
        classes_num = 2 #0 ve 1 olmak üzere 2 sınıfımız var daha fazla olursa ona göre yapılır.
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num)) #one-hot ta 0-> [1,0] ve 1->[0,1] şeklinde gösterilecek 
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        #bu fonksiyon inputs batch ve one-hot encoded targets döndürür
        return inputs_batch, targets_one_hot
    
    #Tüm batchleri tekrar etmek için bir method gerekir ki loopa sokalım
    #Bu bize pythonda sınıfların iterable olduğunu söyler. şu şekilde kullanabiliriz yani:
    #for input, output in data:
    #  do things
    #Pythonda iterasyon için kullanılan sınıf __next__ ile kullanılır çünkü next buna hangi objeleri iterate edceğini verir.
    def __iter__(self):
        return self

### Outlining the Model Algorithm
50 hidden unitli iki hiddn layer tanımlaycağız. input 10(10 sütunla işlem yapıyorduk) output 2 layer

In [2]:
import tensorflow as tf

#mnistte epoch loopa kadar olan kısmı kopyaladık buraya koyduk.

input_size=10                              
output_size=2
hidden_layer_size=100

tf.reset_default_graph()
#bu kod önceki run dan hafızada kalan değerleri resetlemek için kullanılır
#w , b değerlerini güncelledikçe tekrar çalıştıracağımızdan buna ihtiyaç var.

inputs=tf.placeholder(tf.float32,[None,input_size])
targets=tf.placeholder(tf.int32,[None,output_size])

#3 tane w ve b değeri olduğundan(2 hidden unit ve input output araları) ayrı ayrı giriyoruz
weights_1=tf.get_variable("weights_1",[input_size,hidden_layer_size]) #size ı 784*50
biases_1=tf.get_variable("biases_1",[hidden_layer_size])#size ı 50
#değişkenleri atamanın geleneksel bir yoludur bu. ayrıca default olduğundan Xavier initializer kullanır.
#name ve shape parametreleridir

#burada hidden layer 1 değerlerini buluyoruz
outputs_1=tf.nn.relu(tf.matmul(inputs,weights_1)+biases_1) # relu aktivasyon fonksiyonunu kullandık.(nonliearity)
#tf.nn neural network ün kısaltması. yaygın kullanılan aktivasyonn fonksiyonlarını barındırır.

#1. işlem tamam şimdi sıra 2.de

weights_2=tf.get_variable("weights_2",[hidden_layer_size,hidden_layer_size])
biases_2=tf.get_variable("biases_2", [hidden_layer_size])

outputs_2=tf.nn.sigmoid(tf.matmul(outputs_1,weights_2)+biases_2)

weights_3=tf.get_variable("weights_3", [hidden_layer_size,output_size])
biases_3=tf.get_variable("biases_3",[output_size])

outputs=tf.matmul(outputs_2,weights_3)+biases_3

loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs,labels=targets)

#loss var ama mean loss ile çalışmak performansı artırır
#kullanılan fonksiyon boyutlar halindeki tensorun ortalamasını bulur

mean_loss=tf.reduce_mean(loss)

#optimizason fonksiyonu belirlenir.öncekinde gd kullanmıştık ama artık adam kullanacağız

optimize= tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

#output ile targetı karşılaştırıyoruz
#tf.equal eşit mi değil mi değeri döndürür t/f
#tf.argmax fonksiyonu içine verilen değer matrisinin neresinin karşılaştırılacağını alır.outpus,1 derse output matrisinin sütunu, output,0 derse satırı olur
#out_equals_target da her gözlem için 0-1 lerden oluşan bir vektördür.
#accuracy ise bu out_equals_target vektörünün meanidir.bu vektör boolean olduğundan sonucta hata verdirebilir. onun için bunu tf.cast ile floata çeviriyoruz
#tf.cast(object,data_type) bir objeyi başka bir data tipine dönüştürmeye yarar

out_equals_target=tf.equal(tf.argmax(outputs,1),tf.argmax(targets,1))

accuracy=tf.reduce_mean(tf.cast(out_equals_target,tf.float32))
# bu iki satır modelin training accuracy değerini 0-1 arasında olacak şekilde hesapladı.

#session 
sess=tf.InteractiveSession()
initializer=tf.global_variables_initializer()
sess.run(initializer)

##### BATCHİNG ########## BATCHİNGİ CLASS DA YAPIYORUZ BURADA SADECE SİZE VERİLDİ
#batch size =1 olursa SGD olmuş olur. Batch size= number of samples olursa da GD olur.

batch_size=500 #örnek sayısını batch size a bölünce batch sayısı çıkar

########## EARLY STOPPING MEKANİZMALARI########

#ilk olarak algoritmanın train edeceği maximum epoch sayısını giriyoruz
#early stopping eğer validation loss artıyorsa 

max_epochs=1500 # makul zamanda bitirmesi için 15 girdik
prev_validation_loss= 9999999 #yeterince büyük verdik ki 1. epochta hemen bitmesin diye

########### VERİ YÜKLEME #############
# mnistte veriyi yüklerken tensorflow data provider kullanmıştık. bize preprocessed vaziyette getirmişti.
# ama şimdi kendimiz preprocess yaptık. bu yüzden biz yükleyeceğiz datayı.

#Bunlar Audiobooks_data_reader classımızın birer nesnesi
train_data = Audiobooks_Data_Reader('train', batch_size)
validation_data = Audiobooks_Data_Reader('validation')

for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0.
    
    ############# TRAINING ###########
    
    #class iterable olduğundan classı döngüye sokuyoruz
    for input_batch, target_batch in train_data:
        #bu sayede her seferinde train setten 100 er input ve output batchler alacağız
        _, batch_loss=sess.run([optimize,mean_loss],
            feed_dict={inputs:input_batch,targets:target_batch})
        
        curr_epoch_loss += batch_loss
    
     #mean curr_epoch hsaplanır
    curr_epoch_loss /= train_data.batch_count #train_data.batch_count number of batches  
    
    ############ VALIDATING ##############
    
    #forward propagate yaparken loopa gerek yoktu aslında ama class iterator olduğundan böyle olmalı
    #dolayısıyla bu döngü hep tek iterasyon yapar
    for input_batch, target_batch in validation_data:
        validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
                                                feed_dict={inputs:input_batch, targets:target_batch})

######## YAZDIRMA ############
    
    print("Epoch: " +str(epoch_counter+1)+
          ". Training loss: "+" {0:.3f}".format(curr_epoch_loss)+
          ". Validation loss: "+" {0:.3f}".format(validation_loss)+
          ". Validation accuracy: "+"{0:.2f}".format(validation_accuracy*100.)+"%"
         )
    
    
    #### EARLY STOPPING MEKANİZMASI #########
    if validation_loss > prev_validation_loss:
        print("overfitting!!!!!!!")
        break
        
    prev_validation_loss=validation_loss #böylece validation loss artmaya başlayınca duracak
    
print("Training sonu")

Epoch: 1. Training loss:  0.700. Validation loss:  0.642. Validation accuracy: 69.57%
Epoch: 2. Training loss:  0.630. Validation loss:  0.621. Validation accuracy: 61.74%
Epoch: 3. Training loss:  0.593. Validation loss:  0.574. Validation accuracy: 70.92%
Epoch: 4. Training loss:  0.557. Validation loss:  0.544. Validation accuracy: 77.18%
Epoch: 5. Training loss:  0.529. Validation loss:  0.517. Validation accuracy: 75.17%
Epoch: 6. Training loss:  0.503. Validation loss:  0.495. Validation accuracy: 73.60%
Epoch: 7. Training loss:  0.481. Validation loss:  0.474. Validation accuracy: 75.39%
Epoch: 8. Training loss:  0.462. Validation loss:  0.455. Validation accuracy: 77.85%
Epoch: 9. Training loss:  0.446. Validation loss:  0.441. Validation accuracy: 77.85%
Epoch: 10. Training loss:  0.433. Validation loss:  0.429. Validation accuracy: 78.08%
Epoch: 11. Training loss:  0.422. Validation loss:  0.419. Validation accuracy: 79.19%
Epoch: 12. Training loss:  0.412. Validation loss:  

### Testing

In [3]:
test_data = Audiobooks_Data_Reader('test')

for input_batch, target_batch in test_data:
    test_accuracy=sess.run([accuracy],
                      feed_dict={inputs:input_batch, targets:target_batch})
# run ile bir single output yürütürsek sess.run sonucu tek elemanlı list olur

test_accuracy_percent = test_accuracy[0]*100.

print("Test accuracy: " + "{0:.2f}".format(test_accuracy_percent)+"%")

Test accuracy: 79.91%
